In [1]:
#Import libraries in python3 kernel
import pandas as pd
import seaborn as sns
import glob
import os
import sys
!conda install --yes --prefix {sys.prefix} boto
import boto
import shutil
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.colors import LogNorm
import numpy as np
import skbio
#import fastcluster #this package makes skbio run faster clustermaps but can be tricky with missing values from pairwise comparisons
from functools import reduce
!conda install --yes --prefix {sys.prefix} biopython
from Bio import SeqIO
from Bio.SeqIO.FastaIO import SimpleFastaParser
from Bio.SeqUtils import GC
from collections import defaultdict
from collections import Counter
import statistics
import itertools as it
from scipy import stats
from matplotlib.ticker import FormatStrFormatter
from qiime2 import Artifact
import tempfile
import zipfile
import yaml
%matplotlib inline

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [2]:
exp16 = pd.read_csv('expected_16s.tsv', sep='\t')

In [3]:
exp18 = pd.read_csv('expected_18s.tsv', sep='\t')

In [41]:
silvaexp = pd.read_csv('expected_taxonomy.tsv', sep='\t')

In [42]:
silvaexp[['Domain', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']] = silvaexp['Taxon'].str.split(r'; [A-Za-z]__', expand=True)

In [45]:
silvaexp['Domain'] = silvaexp['Domain'].str.split('d__').str[1];

In [46]:
silvaexp

,Feature ID,Taxon,Confidence,Domain,Phylum,Class,Order,Family,Genus,Species
0,7ece9da8c6157d1614c8f73a10beda10fb13a513,d__Bacteria; p__Proteobacteria; c__Alphaproteo...,0.879390,Bacteria,Proteobacteria,Alphaproteobacteria,SAR11_clade,Clade_I,Clade_Ia,None
1,cccf62747bbeaf519bcb9d677ca9b76201ec041b,d__Bacteria; p__Actinobacteriota; c__Acidimicr...,1.000000,Bacteria,Actinobacteriota,Acidimicrobiia,Actinomarinales,Actinomarinaceae,Candidatus_Actinomarina,None
2,b75078d169b134c5b50dff8b8e5063e46a371c15,d__Eukaryota; p__Prymnesiophyceae; c__Prymnesi...,0.918500,Eukaryota,Prymnesiophyceae,Prymnesiophyceae,Prymnesiales,Prymnesiales,Chrysochromulina,uncultured_eukaryote
3,70c10dcffb6a8a32ed6d84d8513126bb42c95d27,d__Eukaryota; p__Protalveolata; c__Syndiniales...,0.982326,Eukaryota,Protalveolata,Syndiniales,Syndiniales,Syndiniales,Amoebophrya,eukaryote_clone
4,7a080ee9e590160d90438981d9acb42d68c1221d,d__Eukaryota; p__Diatomea; c__Coscinodiscophyt...,0.998294,Eukaryota,Diatomea,Coscinodiscophytina,Coscinodiscophytina,Rhizosolenids,Guinardia,Guinardia_striata
5,52a7f08fbb6948eb5520ccaaf8d7d396e25184a2,d__Archaea; p__Crenarchaeota; c__Nitrososphaer...,0.999219,Archaea,Crenarchaeota,Nitrososphaeria,Nitrosopumilales,Nitrosopumilaceae,Candidatus_Nitrosopumilus,None
6,7e17b5d4b0d466ca1623c26e2004a4f50b1dd5c5,d__Bacteria; p__Actinobacteriota; c__Acidimicr...,1.000000,Bacteria,Actinobacteriota,Acidimicrobiia,Actinomarinales,Actinomarinaceae,Candidatus_Actinomarina,None
7,2ecc15ccb808d9513457c0a6edece047e0fd2f8d,d__Eukaryota; p__Dinoflagellata; c__Dinophycea...,0.986416,Eukaryota,Dinoflagellata,Dinophyceae,Gonyaulacales,Gonyaulacales,Lingulodinium,Lingulodinium_polyedrum
8,622145cf3e6086536fd62ce97cedf2ac09504348,d__Eukaryota; p__Cercozoa; c__Thecofilosea; o_...,0.831543,Eukaryota,Cercozoa,Thecofilosea,Cryomonadida,Protaspidae,Cryothecomonas,None
9,a952e69390ba0693a42968ba868bab73816ce655,d__Eukaryota; p__Protalveolata; c__Syndiniales...,0.862756,Eukaryota,Protalveolata,Syndiniales,Syndiniales,Syndiniales_Group_II,Syndiniales_Group_II,uncultured_Syndiniales


In [49]:
expected = pd.merge(exp16,exp18, on=['taxonomy','#OTU ID'], how='outer')

In [51]:
expected = expected.fillna(0)

In [52]:
expected[['Domain', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']] = expected['taxonomy'].str.split(r'; [A-Za-z]_[0-9]__', expand=True)

In [54]:
expected['Domain'] = expected['Domain'].str.split('D_0__').str[1];

In [94]:
expected = expected.rename(columns={'16s-mock-even-insilico-v2': 'exp_16s_even', '16s-mock-staggered-insilico-v2': 'exp_16s_stagg', '18s-mock-even-insilico':'exp_18s_even', '18s-mock-staggered-insilico':'exp_18s_stagg'})

In [104]:
expected16seven = expected[(expected.exp_16s_even > 0)]

In [97]:
expected16stagg = expected[(expected.exp_16s_stagg > 0)]

In [98]:
expected18seven = expected[(expected.exp_18s_even > 0)]

In [99]:
expected18stagg = expected[(expected.exp_18s_even > 0)]

In [100]:
expected16seven = pd.merge(expected16seven,silvaexp, on=['Domain', 'Phylum', 'Class'], how='outer')

In [102]:
expected16seven.to_csv('exp16sevene.csv')

In [110]:
exp_curated= pd.read_csv('exp16sevene.tsv', sep='\t')

In [112]:
exp_16even = exp_curated[['Feature ID', 'Taxon', 'Confidence']].copy()

In [114]:
exp_16even.to_csv('02-PROKs/all_taxonomies/Even46/expected_16even.tsv', sep='\t')

In [67]:
expected_silva.to_csv('expected.csv')

In [63]:
expected_silva_grped = expected_silva.groupby(['Domain', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species'])

In [64]:
expected_silva_grped

In [66]:
expected_silva=expected_silva.sort_values(by='Family')

In [ ]:
top_cols = ['#OTU ID', 'Feature ID', 'taxonomy', 'Taxon']
df = df[[c for c in df if c not in cols_at_end] 
        + [c for c in cols_at_end if c in df]]

In [75]:
df = expected_silva[['#OTU ID', 'Feature ID', '16s-mock-even-insilico-v2', '16s-mock-staggered-insilico-v2', '18s-mock-even-insilico', '18s-mock-staggered-insilico', 'Domain', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species', 'Confidence']]

In [72]:
df.to_csv('df.csv')

In [79]:
df

,#OTU ID,Feature ID,16s-mock-even-insilico-v2,16s-mock-staggered-insilico-v2,18s-mock-even-insilico,18s-mock-staggered-insilico,Domain,Phylum,Class,Order,Family,Genus,Species,Confidence
10,0b1c2b9ed02095f223b442c9bdd35006,NaN,0.0,100.0,0.0,0.0,Bacteria,Proteobacteria,Alphaproteobacteria,Rhodospirillales,AEGEAN-169 marine group,uncultured marine bacterium,uncultured marine bacterium,NaN
2,bdf18e8980906003f3978e5ee578c9ea,NaN,10.0,700.0,0.0,0.0,Bacteria,Actinobacteria,Acidimicrobiia,Actinomarinales,Actinomarinaceae,Candidatus Actinomarina,uncultured marine bacterium,NaN
3,9500b773e54b07b2f96febe7496558fe,NaN,0.0,0.0,0.0,0.0,Bacteria,Actinobacteria,Acidimicrobiia,Actinomarinales,Actinomarinaceae,Candidatus Actinomarina,uncultured marine bacterium,NaN
6,beb8cb17f8152e488b6bc95bfcdb484f,NaN,0.0,400.0,0.0,0.0,Bacteria,Actinobacteria,Acidimicrobiia,Actinomarinales,Actinomarinaceae,Candidatus Actinomarina; Ambiguous_taxa,None,NaN
35,650557dd569e9ca81248b7e08b206a76,NaN,0.0,0.0,10.0,680.0,Eukaryota,Haptophyta,Prymnesiophyceae,Prymnesiales,Chrysochromulina,uncultured eukaryote,uncultured eukaryote,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,444178788694c8d26d44ab43e38ce3e7,NaN,0.0,0.0,0.0,0.0,Eukaryota,SAR,Alveolata,Dinoflagellata,None,None,None,NaN
71,c5c008321c3c0f58014963510712a134,NaN,0.0,0.0,0.0,0.0,Eukaryota,SAR,Stramenopiles,Ochrophyta,None,None,None,NaN
73,5e9fa4cdf604dad2647af697f9c13fbb,NaN,0.0,0.0,0.0,0.0,Eukaryota,SAR,Rhizaria,Retaria,None,None,None,NaN
76,c37a3ea4daf888b4315b155dfb83a3c2,NaN,0.0,0.0,0.0,0.0,Eukaryota,None,None,None,None,None,None,NaN


In [85]:
df = df.rename(columns={'16s-mock-even-insilico-v2': 'exp_16s_even', '16s-mock-staggered-insilico-v2': 'exp_16s_stagg', '18s-mock-even-insilico':'exp_18s_even', '18s-mock-staggered-insilico':'exp_18s_stagg'})

In [86]:
df

,#OTU ID,Feature ID,exp_16s_even,exp_16s_stagg,exp_18s_even,exp_18s_stagg,Domain,Phylum,Class,Order,Family,Genus,Species,Confidence
10,0b1c2b9ed02095f223b442c9bdd35006,NaN,0.0,100.0,0.0,0.0,Bacteria,Proteobacteria,Alphaproteobacteria,Rhodospirillales,AEGEAN-169 marine group,uncultured marine bacterium,uncultured marine bacterium,NaN
2,bdf18e8980906003f3978e5ee578c9ea,NaN,10.0,700.0,0.0,0.0,Bacteria,Actinobacteria,Acidimicrobiia,Actinomarinales,Actinomarinaceae,Candidatus Actinomarina,uncultured marine bacterium,NaN
3,9500b773e54b07b2f96febe7496558fe,NaN,0.0,0.0,0.0,0.0,Bacteria,Actinobacteria,Acidimicrobiia,Actinomarinales,Actinomarinaceae,Candidatus Actinomarina,uncultured marine bacterium,NaN
6,beb8cb17f8152e488b6bc95bfcdb484f,NaN,0.0,400.0,0.0,0.0,Bacteria,Actinobacteria,Acidimicrobiia,Actinomarinales,Actinomarinaceae,Candidatus Actinomarina; Ambiguous_taxa,None,NaN
35,650557dd569e9ca81248b7e08b206a76,NaN,0.0,0.0,10.0,680.0,Eukaryota,Haptophyta,Prymnesiophyceae,Prymnesiales,Chrysochromulina,uncultured eukaryote,uncultured eukaryote,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,444178788694c8d26d44ab43e38ce3e7,NaN,0.0,0.0,0.0,0.0,Eukaryota,SAR,Alveolata,Dinoflagellata,None,None,None,NaN
71,c5c008321c3c0f58014963510712a134,NaN,0.0,0.0,0.0,0.0,Eukaryota,SAR,Stramenopiles,Ochrophyta,None,None,None,NaN
73,5e9fa4cdf604dad2647af697f9c13fbb,NaN,0.0,0.0,0.0,0.0,Eukaryota,SAR,Rhizaria,Retaria,None,None,None,NaN
76,c37a3ea4daf888b4315b155dfb83a3c2,NaN,0.0,0.0,0.0,0.0,Eukaryota,None,None,None,None,None,None,NaN


In [88]:
expected16seven = df[(df.exp_16s_even > 0)]

In [89]:
expected16seven

,#OTU ID,Feature ID,exp_16s_even,exp_16s_stagg,exp_18s_even,exp_18s_stagg,Domain,Phylum,Class,Order,Family,Genus,Species,Confidence
2,bdf18e8980906003f3978e5ee578c9ea,NaN,10.0,700.0,0.0,0.0,Bacteria,Actinobacteria,Acidimicrobiia,Actinomarinales,Actinomarinaceae,Candidatus Actinomarina,uncultured marine bacterium,NaN
0,863728e1cec6befd5ba02d15baef4c36,NaN,10.0,1400.0,0.0,0.0,Bacteria,Proteobacteria,Alphaproteobacteria,SAR11 clade,Clade I,None,None,NaN
7,86cd70d2450e3f44f4a7c543f53008ee,NaN,10.0,300.0,0.0,0.0,Bacteria,Cyanobacteria,Oxyphotobacteria,Synechococcales,Cyanobiaceae,Prochlorococcus MIT9313,None,NaN
16,20a0d2f150b3312772f5c00e54fd7baf,NaN,10.0,80.0,0.0,0.0,Bacteria,Bacteroidetes,Bacteroidia,Flavobacteriales,Flavobacteriaceae,NS2b marine group,None,NaN
4,2970af3e30642198ba1dfa1f304f310b,NaN,10.0,400.0,0.0,0.0,Archaea,Thaumarchaeota,Nitrososphaeria,Nitrosopumilales,Nitrosopumilaceae,Candidatus Nitrosopumilus; Ambiguous_taxa,None,NaN
11,9c0fe787b120b62b93c3cb46d844d9fa,NaN,10.0,80.0,0.0,0.0,Bacteria,Planctomycetes,Planctomycetacia,Pirellulales,Pirellulaceae,Blastopirellula,uncultured Pirellula sp.,NaN
8,f4ad315ec51d8bee23cb39f39bc8b019,NaN,10.0,200.0,0.0,0.0,Bacteria,Proteobacteria,Gammaproteobacteria,SAR86 clade,Rhodobacteraceae bacterium REDSEA-S29_B10,Rhodobacteraceae bacterium REDSEA-S29_B10,Rhodobacteraceae bacterium REDSEA-S29_B10,NaN
9,b8b0859fd6afcfc39c1a7e6a50415ca3,NaN,10.0,100.0,0.0,0.0,Bacteria,Proteobacteria,Alphaproteobacteria,Puniceispirillales,SAR116 clade,uncultured marine bacterium,uncultured marine bacterium,NaN
20,167b5b3eeed0f9b502f7f1f245d5116c,NaN,10.0,60.0,0.0,0.0,Bacteria,Marinimicrobia (SAR406 clade),uncultured marine bacterium,uncultured marine bacterium,uncultured marine bacterium,uncultured marine bacterium,uncultured marine bacterium,NaN
15,0b7809b1b04598cfc5bbf6264159dde9,NaN,10.0,80.0,0.0,0.0,Archaea,Euryarchaeota,Thermoplasmata,Marine Group II,None,None,None,NaN


In [90]:
expected16seven=expected16seven.drop(columns=['exp_16s_stagg', 'exp_18s_even', 'exp_18s_stagg'])

,#OTU ID,Feature ID,exp_16s_even,Domain,Phylum,Class,Order,Family,Genus,Species,Confidence
2,bdf18e8980906003f3978e5ee578c9ea,NaN,10.0,Bacteria,Actinobacteria,Acidimicrobiia,Actinomarinales,Actinomarinaceae,Candidatus Actinomarina,uncultured marine bacterium,NaN
0,863728e1cec6befd5ba02d15baef4c36,NaN,10.0,Bacteria,Proteobacteria,Alphaproteobacteria,SAR11 clade,Clade I,None,None,NaN
7,86cd70d2450e3f44f4a7c543f53008ee,NaN,10.0,Bacteria,Cyanobacteria,Oxyphotobacteria,Synechococcales,Cyanobiaceae,Prochlorococcus MIT9313,None,NaN
16,20a0d2f150b3312772f5c00e54fd7baf,NaN,10.0,Bacteria,Bacteroidetes,Bacteroidia,Flavobacteriales,Flavobacteriaceae,NS2b marine group,None,NaN
4,2970af3e30642198ba1dfa1f304f310b,NaN,10.0,Archaea,Thaumarchaeota,Nitrososphaeria,Nitrosopumilales,Nitrosopumilaceae,Candidatus Nitrosopumilus; Ambiguous_taxa,None,NaN
11,9c0fe787b120b62b93c3cb46d844d9fa,NaN,10.0,Bacteria,Planctomycetes,Planctomycetacia,Pirellulales,Pirellulaceae,Blastopirellula,uncultured Pirellula sp.,NaN
8,f4ad315ec51d8bee23cb39f39bc8b019,NaN,10.0,Bacteria,Proteobacteria,Gammaproteobacteria,SAR86 clade,Rhodobacteraceae bacterium REDSEA-S29_B10,Rhodobacteraceae bacterium REDSEA-S29_B10,Rhodobacteraceae bacterium REDSEA-S29_B10,NaN
9,b8b0859fd6afcfc39c1a7e6a50415ca3,NaN,10.0,Bacteria,Proteobacteria,Alphaproteobacteria,Puniceispirillales,SAR116 clade,uncultured marine bacterium,uncultured marine bacterium,NaN
20,167b5b3eeed0f9b502f7f1f245d5116c,NaN,10.0,Bacteria,Marinimicrobia (SAR406 clade),uncultured marine bacterium,uncultured marine bacterium,uncultured marine bacterium,uncultured marine bacterium,uncultured marine bacterium,NaN
15,0b7809b1b04598cfc5bbf6264159dde9,NaN,10.0,Archaea,Euryarchaeota,Thermoplasmata,Marine Group II,None,None,None,NaN
